# Pedestrian Classification with Normalized HOG Features

This notebook implements pedestrian classification using normalized HOG (Histogram of Oriented Gradients) features.

In [ ]:
import os, cv2, joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [ ]:
# Create directories
os.makedirs("Models", exist_ok=True)
os.makedirs("Results", exist_ok=True)

In [ ]:
def load_pos(folder):
    """Load positive samples (pedestrians)"""
    imgs = []
    for f in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, f))
        if img is not None:
            imgs.append(cv2.resize(img, (64, 128)))
    return imgs

def load_neg(folder, n=5):
    """Load negative samples (non-pedestrians)"""
    imgs = []
    for f in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, f))
        if img is None: 
            continue
        h, w, _ = img.shape
        for _ in range(n):
            x = np.random.randint(0, w-64)
            y = np.random.randint(0, h-128)
            imgs.append(img[y:y+128, x:x+64])
    return imgs

In [ ]:
# Load data
train_pos = load_pos("../INRIAPerson/Train/pos")
train_neg = load_neg("../INRIAPerson/Train/neg")

val_pos = load_pos("../INRIAPerson/Validation/pos")
val_neg = load_neg("../INRIAPerson/Validation/neg")

X_train_imgs = train_pos + train_neg
y_train = np.array([1]*len(train_pos) + [0]*len(train_neg))

X_val_imgs = val_pos + val_neg
y_val = np.array([1]*len(val_pos) + [0]*len(val_neg))

print(f"Train samples: {len(X_train_imgs)}")
print(f"Validation samples: {len(X_val_imgs)}")

In [ ]:
# Visualize sample images
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

# Show positive samples
for i in range(2):
    img = cv2.cvtColor(train_pos[i], cv2.COLOR_BGR2RGB)
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'Positive {i+1}')
    axes[0, i].axis('off')

# Show negative samples
for i in range(2):
    img = cv2.cvtColor(train_neg[i], cv2.COLOR_BGR2RGB)
    axes[1, i].imshow(img)
    axes[1, i].set_title(f'Negative {i+1}')
    axes[1, i].axis('off')

# Show HOG visualizations
for i in range(2):
    gray_pos = cv2.cvtColor(train_pos[i], cv2.COLOR_BGR2GRAY)
    gray_neg = cv2.cvtColor(train_neg[i], cv2.COLOR_BGR2GRAY)
    
    _, hog_pos = hog(gray_pos, orientations=9, pixels_per_cell=(8, 8),
                     cells_per_block=(2, 2), visualize=True)
    _, hog_neg = hog(gray_neg, orientations=9, pixels_per_cell=(8, 8),
                     cells_per_block=(2, 2), visualize=True)
    
    axes[0, i+2].imshow(hog_pos, cmap='gray')
    axes[0, i+2].set_title(f'HOG Pos {i+1}')
    axes[0, i+2].axis('off')
    
    axes[1, i+2].imshow(hog_neg, cmap='gray')
    axes[1, i+2].set_title(f'HOG Neg {i+1}')
    axes[1, i+2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def extract_hog_features(images):
    """Extract HOG features from images"""
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hog_feat = hog(gray, orientations=9, pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), visualize=False)
        features.append(hog_feat)
    return np.array(features)

In [ ]:
# Extract HOG features
print("Extracting HOG features...")
X_train_hog = extract_hog_features(X_train_imgs)
X_val_hog = extract_hog_features(X_val_imgs)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_hog)
X_val = scaler.transform(X_val_hog)
joblib.dump(scaler, "Models/scaler.joblib")


print(f"Feature shape: {X_train.shape}")

In [ ]:
# Define models with optimized hyperparameters
models = {
    'SVM': SVC(
        C=1,
        kernel='rbf', 
        gamma='scale', 
        probability=True, 
        random_state=42
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(100,), 
        activation='relu', 
        solver='adam', 
        alpha=0.001, 
        learning_rate='constant', 
        max_iter=1000, 
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=500, 
        max_depth=3, 
        learning_rate=0.1, 
        subsample=1.0, 
        random_state=42
    ),
    'Logistic Regression': LogisticRegression(
        C=0.01, 
        solver='lbfgs', 
        penalty='l2', 
        max_iter=2000, 
        random_state=42
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100, # You can adjust based on your specific RF result
        random_state=42
    ),
    'KNN': KNeighborsClassifier(
        n_neighbors=3, 
        weights='distance', 
        metric='manhattan'
    )
}

In [ ]:
# ====================================================
# Train and Evaluate Models
# ====================================================

results = []

for name, model in models.items():
    print(f"Training Optimized {name}...")
    
    # Train model (using your pre-scaled X_train)
    model.fit(X_train, y_train)
    
    # Predictions
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    # Accuracies
    train_acc = accuracy_score(y_train, train_pred)
    val_acc = accuracy_score(y_val, val_pred)
    
    results.append({
        'Model': name,
        'Train_Acc': train_acc,
        'Val_Acc': val_acc,
        'Overfit_Gap': train_acc - val_acc
    })
    
    # Save the final optimized model
    joblib.dump(model, f"Models/{name.replace(' ', '_')}_final.joblib")
    
    print(f"{name} Results -> Train: {train_acc:.4f}, Val: {val_acc:.4f}")

# ====================================================
# Save Final Comparison
# ====================================================

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Val_Acc', ascending=False)
results_df.to_csv('Results/final_model_performance.csv', index=False)

print("\n" + "="*30)
print("FINAL PERFORMANCE SUMMARY")
print("="*30)
print(results_df)

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
x_pos = np.arange(len(results_df))

plt.bar(x_pos - 0.2, results_df['Train_Acc'], 0.4, label='Train Accuracy', alpha=0.8)
plt.bar(x_pos + 0.2, results_df['Val_Acc'], 0.4, label='Validation Accuracy', alpha=0.8)

plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.xticks(x_pos, results_df['Model'], rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('Results/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Best model analysis
best_model_name = results_df.iloc[0]['Model']
best_model = joblib.load(f"Models/{best_model_name.replace(' ', '_')}.joblib")

print(f"Best Model: {best_model_name}")
print(f"Validation Accuracy: {results_df.iloc[0]['Val_Acc']:.4f}")

# Confusion matrix
val_pred = best_model.predict(X_val)
cm = confusion_matrix(y_val, val_pred)

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Non-Pedestrian', 'Pedestrian'])
plt.yticks(tick_marks, ['Non-Pedestrian', 'Pedestrian'])

# Add text annotations
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('Results/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_val, val_pred, target_names=['Non-Pedestrian', 'Pedestrian']))

In [ ]:
# ====================================================
# Step 10 — Test All Models on HOG_NORMALIZED Test Set
# ====================================================

# Load test data
test_pos = load_pos("../INRIAPerson/Test/pos")
test_neg = load_neg("../INRIAPerson/Test/neg")

X_test_imgs = test_pos + test_neg
y_test = np.array([1]*len(test_pos) + [0]*len(test_neg))

print(f"Test samples: {len(X_test_imgs)}")

# Extract HOG features for test set
X_test_hog = extract_hog_features(X_test_imgs)

# Load scaler for normalized features
scaler = joblib.load("Models/scaler.joblib")  # Save this during training

# Normalize test features
X_test_norm = scaler.transform(X_test_hog)

# List of model names
model_names = ["SVM", "Random_Forest", "KNN", "Logistic_Regression", "Gradient_Boosting", "Neural_Network"]

# Loop through each model
for name in model_names:
    print(f"\nTesting Model: {name}")
    
    # Load trained model
    model = joblib.load(f"Models/{name}.joblib")
    
    # Predict
    test_pred = model.predict(X_test_norm)
    test_acc = accuracy_score(y_test, test_pred)
    
    print(f"{name} Test Accuracy: {test_acc:.4f}")
    print(classification_report(y_test, test_pred, target_names=['Negative','Positive']))
    
    # Find correct and wrong predictions
    correct_idx = np.where(test_pred == y_test)[0]
    wrong_idx = np.where(test_pred != y_test)[0]
    
    # Randomly select 6 from each
    correct_sample = np.random.choice(correct_idx, min(6, len(correct_idx)), replace=False)
    wrong_sample = np.random.choice(wrong_idx, min(6, len(wrong_idx)), replace=False)
    
    # --- Visualize correct predictions ---
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()
    for i, idx in enumerate(correct_sample):
        img = cv2.cvtColor(X_test_imgs[idx], cv2.COLOR_BGR2RGB)
        true_label = 'Pedestrian' if y_test[idx] == 1 else 'Non-Pedestrian'
        axes[i].imshow(img)
        axes[i].set_title(f'True: {true_label}', color='green', fontsize=10)
        axes[i].axis('off')
    plt.suptitle(f"{name} — 6 Correct Predictions", fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # --- Visualize wrong predictions ---
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()
    for i, idx in enumerate(wrong_sample):
        img = cv2.cvtColor(X_test_imgs[idx], cv2.COLOR_BGR2RGB)
        true_label = 'Pedestrian' if y_test[idx] == 1 else 'Non-Pedestrian'
        pred_label = 'Pedestrian' if test_pred[idx] == 1 else 'Non-Pedestrian'
        axes[i].imshow(img)
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}', color='red', fontsize=10)
        axes[i].axis('off')
    plt.suptitle(f"{name} — 6 Wrong Predictions", fontsize=16)
    plt.tight_layout()
    plt.show()
